In [1]:
import pandas as pd
import numpy
import geopandas as gpd
# from dbfread import DBF
from simpledbf import Dbf5
import os, sys

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


### I/O

In [2]:
# INPUT
# Network that contains all-lane tolling
NETWORK_DIR = 'L:\\Application\\Model_One\\NextGenFwys\\Scenarios\\2035_TM152_NGF_NP02_BPALTsegmented_00\\OUTPUT\\shapefile'
ROADWAY_FILE = os.path.join(NETWORK_DIR, 'network_links.dbf')

# nodes on arterial projects
ART_PROJ_DIR = 'M:\\Application\\Model One\\NetworkProjects'

# transit network
# TEMP_DIR = 'L:\\Application\\Model_One\\NextGenFwys\\Scenarios\\2035_TM152_NGF_NP02_BPALTsegmented_00\\OUTPUT\\shapefile - no crowding'
TRN_LINKS_FILE = os.path.join(NETWORK_DIR, 'network_trn_links.shp')
TRN_ROUTE_LINKS_FILE = os.path.join(NETWORK_DIR, 'network_trn_route_links.shp')
TRN_STOPS_FILE = os.path.join(NETWORK_DIR, 'network_trn_stops.shp')
TRN_LINES_FILE = os.path.join(NETWORK_DIR, 'network_trn_lines.shp')

# OUTPUT
OUTPUT_DIR = os.path.join(NETWORK_DIR, 'for_transitCrowding_tableau')

In [3]:
dbf = Dbf5(ROADWAY_FILE)
df = dbf.to_dataframe()
print(list(df))

['A', 'B', 'DISTANCE', 'SPDCLASS', 'CAPCLASS', 'LANES', 'TSIN', 'GL', 'OT', 'CAP', 'AT', 'FT', 'SIGCOR', 'TOS', 'AUX', 'YEAR', 'FFS', 'FFT', 'FT2000', 'ROUTENUM', 'HOT', 'TOLLCLASS', 'STATE', 'CITYID', 'CITYNAME', 'REGFREIGHT', 'BRT', 'TOLLEA_DA', 'TOLLEA_S2', 'TOLLEA_S3', 'TOLLEA_VSM', 'TOLLEA_SML', 'TOLLEA_MED', 'TOLLEA_LRG', 'TOLLAM_DA', 'TOLLAM_S2', 'TOLLAM_S3', 'TOLLAM_VSM', 'TOLLAM_SML', 'TOLLAM_MED', 'TOLLAM_LRG', 'TOLLMD_DA', 'TOLLMD_S2', 'TOLLMD_S3', 'TOLLMD_VSM', 'TOLLMD_SML', 'TOLLMD_MED', 'TOLLMD_LRG', 'TOLLPM_DA', 'TOLLPM_S2', 'TOLLPM_S3', 'TOLLPM_VSM', 'TOLLPM_SML', 'TOLLPM_MED', 'TOLLPM_LRG', 'TOLLEV_DA', 'TOLLEV_S2', 'TOLLEV_S3', 'TOLLEV_VSM', 'TOLLEV_SML', 'TOLLEV_MED', 'TOLLEV_LRG', 'ROUTEDIR', 'PROJ', 'AUTOOPC', 'SMTROPC', 'LRTROPC', 'BUSOPC', 'VOT', 'TRUCKVOT', 'SR2COSTSHAR', 'SR3COSTSHAR', 'TRKPCE', 'FIRSTVALUE', 'OWNEDAV_ZPV', 'TNC_ZPV_FAC', 'MEANS_BASED', 'TOLL_FLAT', 'AV_PCE', 'HOVXPEN', '_SEGMENT', '_DIR', 'ACTION', 'PROJECT', 'PNROK', '_PROJECT', 'VOL_S2AV', '

### Get Freeway and Arterial nodes

In [4]:
# get all-lane tolling nodes

# load network
dbf = Dbf5(ROADWAY_FILE)
network_links_df = dbf.to_dataframe()
network_links_df = network_links_df[['A', 'B', 'TOLLCLASS', 'PROJ', 'DISTANCE']]

# tag all-lane-tolling links
network_links_df.loc[network_links_df.TOLLCLASS > 990000, 'freeway'] = True
print('{:,} node pairs, containing the following {} toll classes: {}'.format(
    network_links_df.loc[network_links_df['freeway'] == True].shape[0],
    network_links_df.loc[network_links_df['freeway'] == True].TOLLCLASS.nunique(),
    sorted(network_links_df.loc[network_links_df['freeway'] == True].TOLLCLASS.unique())))

2,377 node pairs, containing the following 121 toll classes: [990075, 990076, 990077, 990078, 990079, 990081, 990082, 990083, 990084, 990085, 990086, 990087, 990088, 990089, 990090, 990091, 990092, 990101, 990102, 990103, 990104, 990105, 990106, 990107, 990108, 990109, 990110, 990111, 990112, 990113, 990114, 990115, 990116, 990117, 990118, 990119, 990120, 990121, 990122, 990123, 990124, 990131, 990132, 990135, 990136, 990137, 990138, 990171, 990172, 990231, 990232, 990233, 990234, 990239, 990240, 990241, 990242, 990243, 990244, 990245, 990246, 990281, 990282, 990283, 990284, 990285, 990286, 990287, 990288, 990289, 990290, 990381, 990382, 990401, 990402, 990403, 990404, 990407, 990408, 990581, 990582, 990583, 990584, 990585, 990586, 990587, 990588, 990589, 990590, 990683, 990684, 990689, 990690, 990691, 990692, 990853, 990854, 990855, 990856, 990871, 990872, 990879, 990880, 990881, 990882, 990883, 990884, 990885, 990886, 990887, 990888, 990889, 990890, 990891, 990892, 990923, 990924, 99

In [5]:
network_links_df.head()

,A,B,TOLLCLASS,PROJ,DISTANCE,freeway
0,1,7487,0,NaN,0.05201,NaN
1,1,7489,0,NaN,0.04758,NaN
2,2,7496,0,NaN,0.06356,NaN
3,2,7500,0,NaN,0.05204,NaN
4,3,7483,0,NaN,0.12645,NaN


In [6]:
# nodes on arterials

# create a dataframe to store the node pairs
artierial_links_df = pd.DataFrame()

# loop through arterial projects to get the node pairs
for proj in os.listdir(ART_PROJ_DIR):
    if ('NGF' in proj) and ('Art' in proj):
        print(proj)
        mod_links_df = pd.read_csv(os.path.join(ART_PROJ_DIR, proj, 'mod_links_tollclass_direction.csv'))
        artierial_links_df = pd.concat([artierial_links_df, mod_links_df])

# rename to be consistent
# artierial_nodes.rename(columns = {'_Project': 'PROJ',
#                                   '_Direction': 'DIRECTION',
#                                   '_Tollclass': 'TOLLCLASS'}, inplace=True)
print('{:,} arterial links, containing the following {} toll classes: {}'.format(
    artierial_links_df.shape[0],
    artierial_links_df['_Tollclass'].nunique(),
    sorted(artierial_links_df['_Tollclass'].unique())))

# add a NGF tag
artierial_links_df['arterial'] = True

NGF_ALA_Art_International_E14_Mission
NGF_ALA_Art_SanPablo
NGF_SC_Art_FoothillExpy
NGF_SC_Art_MontereyRd
NGF_SC_Art_Tasman_NCapitol_ECapitol
NGF_SF_Art_3rd_Bayshore_AirportBlvd
NGF_SM_Art_ElCamino
NGF_SOL_Art_TexasSt
NGF_SON_Art_Mendocino_SantaRosa
773 arterial links, containing the following 18 toll classes: [770080, 770081, 770082, 770083, 770101, 770102, 770103, 770104, 770105, 770106, 770107, 770108, 770237, 770238, 770280, 770281, 770580, 770581]


In [7]:
artierial_links_df

,A,B,_Project,_Direction,_Tollclass,arterial
0,3032,3101,NGF_ALA_Art_International_E14_Mission,S,770581,True
1,3032,12091,NGF_ALA_Art_International_E14_Mission,N,770580,True
2,3040,3041,NGF_ALA_Art_International_E14_Mission,S,770581,True
3,3040,3098,NGF_ALA_Art_International_E14_Mission,N,770580,True
4,3041,3040,NGF_ALA_Art_International_E14_Mission,N,770580,True
...,...,...,...,...,...,...
33,11446,11445,NGF_SON_Art_Mendocino_SantaRosa,S,770108,True
34,12247,8390,NGF_SON_Art_Mendocino_SantaRosa,S,770108,True
35,12247,11445,NGF_SON_Art_Mendocino_SantaRosa,N,770107,True
36,12253,8333,NGF_SON_Art_Mendocino_SantaRosa,N,770107,True


In [8]:
# merge arterial tag to all roadway node pairs
network_links_df = network_links_df.merge(artierial_links_df, on=['A', 'B'], how='left')

# consolidate fields
network_links_df.loc[network_links_df['freeway'] == True, 'link_tag'] = 'freeway'
network_links_df.loc[network_links_df['freeway'] == True, 'Project'] = network_links_df['PROJ']
network_links_df.loc[network_links_df['freeway'] == True, 'TollClass'] = network_links_df['TOLLCLASS']

network_links_df.loc[network_links_df['arterial'] == True, 'link_tag'] = 'arterial'
network_links_df.loc[network_links_df['arterial'] == True, 'Project'] = network_links_df['_Project']
network_links_df.loc[network_links_df['arterial'] == True, 'TollClass'] = network_links_df['_Tollclass']

# keep only freeway and arterial links with needed fields
freeway_arterial_links_df = network_links_df.loc[network_links_df['link_tag'].isin(['freeway', 'arterial'])][[
    'A', 'B', 'DISTANCE', 'Project', 'TollClass', 'link_tag'
]]

In [9]:
network_links_df

,A,B,TOLLCLASS,PROJ,DISTANCE,freeway,_Project,_Direction,_Tollclass,arterial,link_tag,Project,TollClass
0,1,7487,0,NaN,0.05201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7489,0,NaN,0.04758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,7496,0,NaN,0.06356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,7500,0,NaN,0.05204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,7483,0,NaN,0.12645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34391,38792,38793,0,US101_92,0.80000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34392,38793,6359,0,US101_92,0.01000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34393,38802,4042,0,NaN,1.43235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34394,38802,11229,0,NaN,1.19142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
freeway_arterial_links_df

,A,B,DISTANCE,Project,TollClass,link_tag
2705,1602,11993,0.81,BayAreaForward NGF_BlueprintSegmented NGF_Blue...,990092.0,freeway
2708,1603,11944,0.59,BayAreaForward NGF_BlueprintSegmented NGF_Blue...,990091.0,freeway
2756,1617,1618,0.34,BayAreaForward NGF_BlueprintSegmented NGF_Blue...,990401.0,freeway
2759,1618,1619,1.29,BayAreaForward NGF_BlueprintSegmented NGF_Blue...,990401.0,freeway
2762,1619,2155,1.48,BayAreaForward NGF_BlueprintSegmented NGF_Blue...,990401.0,freeway
...,...,...,...,...,...,...
34296,20612,20609,0.32,EXP_uncommitted NGF_BlueprintSegmented,990584.0,freeway
34298,20613,20612,0.57,EXP_uncommitted NGF_BlueprintSegmented,990584.0,freeway
34300,20614,9837,0.52,EXP_uncommitted NGF_BlueprintSegmented,990585.0,freeway
34322,26160,9685,0.64,EXP_uncommitted NGF_BlueprintSegmented,990282.0,freeway


### get transit links' lengths

In [11]:
trn_links_raw = pd.read_csv(r'L:\Application\Model_One\NextGenFwys\Scenarios\2035_TM152_NGF_NP02_BPALTsegmented_00\OUTPUT\trn\trnlink.csv')

trn_links_distance = trn_links_raw.groupby(["A","B","mode","name","timeperiod"]).agg(
    {"distance":numpy.mean,
     "time"    :numpy.mean}).reset_index()

trn_links_distance = trn_links_distance.pivot(index=['A', 'B', 'name'], columns='timeperiod', values='distance').reset_index()
trn_links_distance.fillna(value=0, inplace=True)

In [12]:
trn_links_distance['avg_distance'] = trn_links_distance[['am', 'ea', 'ev', 'md', 'pm']].mean(axis=1)
trn_links_distance.rename(columns = {'am': 'dis_am_raw', 
                                     'ea': 'dis_ea_raw',
                                     'ev': 'dis_ev_raw',
                                     'md': 'dis_md_raw',
                                     'pm': 'dis_pm_raw',
                                     'name': 'NAME'}, inplace=True)

In [13]:
# trn_links_merged.loc[(trn_links_merged['DIST_sum'] == 0) & (trn_links_merged['link_tag'].notnull())][[
#     'A', 'B', 'NAME', 'DISTANCE', 'Project', 'TollClass', 'link_tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']]

### tag transit links and lines based on freeway/arterial nodes

#### trn_links

In [14]:
# transit links shapefile
trn_links_gdf = gpd.read_file(TRN_LINKS_FILE)
print('load {:,} rows of transit links, {:,} unique A+B pair, the following columns: {}'.format(
    trn_links_gdf.shape[0],
    trn_links_gdf[['A', 'B']].drop_duplicates().shape[0],
    list(trn_links_gdf)))
display(trn_links_gdf.head())

load 70,606 rows of transit links, 41,389 unique A+B pair, the following columns: ['Id', 'NAME', 'A', 'B', 'A_STATION', 'B_STATION', 'SEQ', 'NAMESEQAB', 'MODE', 'MODE_NAME', 'MODE_TYPE', 'AB_VOL_EA', 'AB_VOL_AM', 'AB_VOL_MD', 'AB_VOL_PM', 'AB_VOL_EV', 'LOAD_EA', 'LOAD_AM', 'LOAD_MD', 'LOAD_PM', 'LOAD_EV', 'LOADSE_EA', 'LOADSE_AM', 'LOADSE_MD', 'LOADSE_PM', 'LOADSE_EV', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV', 'TIME_EA', 'TIME_AM', 'TIME_MD', 'TIME_PM', 'TIME_EV', 'geometry']


,Id,NAME,A,B,A_STATION,B_STATION,SEQ,NAMESEQAB,MODE,MODE_NAME,...,DIST_AM,DIST_MD,DIST_PM,DIST_EV,TIME_EA,TIME_AM,TIME_MD,TIME_PM,TIME_EV,geometry
0,0,100_ALAFB,14604,14600,Ferry Alameda Ferry Terminal,Ferry SF Ferry Building,1,100_ALAFB 1 14604 14600,100,East Bay Ferries,...,5.6,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 553449.000..."
1,0,100_ALAOAKFB,14604,14605,Ferry Alameda Ferry Terminal,Ferry Jack London Square Ferry Terminal,1,100_ALAOAKFB 1 14604 14605,100,East Bay Ferries,...,0.9,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 563719.000..."
2,0,100_ALAOAKFB,14605,14600,Ferry Jack London Square Ferry Terminal,Ferry SF Ferry Building,2,100_ALAOAKFB 2 14605 14600,100,East Bay Ferries,...,6.4,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,"LINESTRING (563719.000 4183438.000, 553449.000..."
3,0,100_ALOAKSSF,14604,14605,Ferry Alameda Ferry Terminal,Ferry Jack London Square Ferry Terminal,1,100_ALOAKSSF 1 14604 14605,100,East Bay Ferries,...,0.9,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 563719.000..."
4,0,100_ALOAKSSF,14605,14567,Ferry Jack London Square Ferry Terminal,Ferry Oyster Point/South SF,2,100_ALOAKSSF 2 14605 14567,100,East Bay Ferries,...,13.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,0.0,"LINESTRING (563719.000 4183438.000, 554337.312..."


In [15]:
trn_links_gdf.MODE_TYPE.value_counts(dropna=False)

Local Bus                   25044
Express Bus                 14494
Drive access connector       8360
Drive egress connector       8335
Walk egress connector        5420
Walk access connector        5348
Stop-to-stop transfer        1281
Light Rail                    781
Commuter Rail                 415
Walk access funnel link       392
Heavy Rail                    357
Drive access funnel link      262
Ferry                         117
Name: MODE_TYPE, dtype: int64

In [16]:
trn_links_gdf = trn_links_gdf.loc[trn_links_gdf.MODE_TYPE.isin(
    ['Local Bus', 'Express Bus', 'Light Rail', 'Commuter Rail', 'Heavy Rail'])]
print('{:,} rows remain after removing connector/transfer/funnel links and ferry links'.format(trn_links_gdf.shape[0]))

41,091 rows remain after removing connector/transfer/funnel links and ferry links


In [17]:
# merge freeway and arterial nodes
trn_links_merged = trn_links_gdf.merge(freeway_arterial_links_df, on=['A', 'B'], how='left', indicator=True)
print(trn_links_merged['_merge'].value_counts())

left_only     34385
both           6706
right_only        0
Name: _merge, dtype: int64


In [18]:
# highlight 5 sets of links
freeway_idx = (trn_links_merged['_merge'] == 'both') & (trn_links_merged['link_tag'] == 'freeway')
arterial_tag_idx = (trn_links_merged['_merge'] == 'both') & (trn_links_merged['link_tag'] == 'arterial')
heavyRail_tag_idx = (trn_links_merged['_merge'] == 'left_only') & (trn_links_merged['MODE_TYPE'] == 'Heavy Rail')
commuterRail_tag_idx = (trn_links_merged['_merge'] == 'left_only') & (trn_links_merged['MODE_TYPE'] == 'Commuter Rail')

# trn_links_merged = trn_links_merged.loc[freeway_idx | arterial_tag_idx | heavyRail_tag_idx | lightRail_tag_idx | commuterRail_tag_idx]

# tag heavy rail, commuter rail and light rail links (links on freeway and arterial are already tagged)
trn_links_merged.loc[heavyRail_tag_idx, 'link_tag'] = 'heavy rail'
trn_links_merged.loc[commuterRail_tag_idx, 'link_tag'] = 'commuter rail'

# fill na
trn_links_merged['link_tag'].fillna('non-FA', inplace=True)

# create a more aggregated "tag category"
trn_links_merged['link_tag_cat'] = 'non-FA'
trn_links_merged.loc[trn_links_merged['link_tag'].isin(['freeway', 'arterial']), 'link_tag_cat'] = 'FA'
trn_links_merged.loc[trn_links_merged['link_tag'].isin(['heavy rail', 'commuter rail']), 'link_tag_cat'] = trn_links_merged['link_tag']

# drop '_merge'
trn_links_merged.drop(['_merge'], axis=1, inplace=True)

In [19]:
# export
trn_links_merged.to_file('trn_links_NGF_tag.shp')

C:\Users\ywang\AppData\Local\Temp\ipykernel_12820\3453044813.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  trn_links_merged.to_file('trn_links_NGF_tag.shp')


In [20]:
# trn_links_merged = pd.merge(trn_links_merged,
#                             trn_lines_gdf[['NAME', 'NAME_SET', 'OPERATOR_T']],
#                             on='NAME',
#                             how='left')

In [21]:
# trn_links_merged.loc[trn_links_merged.link_tag.isin(['freeway', 'arterial']) & \
#                      (trn_links_merged.DIST_mean == 0) & \
#                      (trn_links_merged.avg_distance !=0) & \
#                      (trn_links_merged.avg_distance.notnull())][[
#     'A', 'B', 'DISTANCE', 'Project', 'TollClass', 'link_tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV', 'DIST_mean','avg_distance']]

#### share of link distance for non-freeway and non-arterial segments

In [22]:
trn_links_merged

,Id,NAME,A,B,A_STATION,B_STATION,SEQ,NAMESEQAB,MODE,MODE_NAME,...,TIME_AM,TIME_MD,TIME_PM,TIME_EV,geometry,DISTANCE,Project,TollClass,link_tag,link_tag_cat
0,0,10_WBerkeley,2588,2650,None,None,1,10_WBerkeley 1 2588 2650,10,West Berkeley,...,0.0,0.0,0.0,0.0,"LINESTRING (564241.000 4189942.000, 564074.000...",NaN,NaN,NaN,non-FA,non-FA
1,0,10_WBerkeley,2650,2661,None,None,2,10_WBerkeley 2 2650 2661,10,West Berkeley,...,0.0,0.0,0.0,0.0,"LINESTRING (564074.000 4189909.000, 563391.000...",NaN,NaN,NaN,non-FA,non-FA
2,0,10_WBerkeley,2661,2662,None,None,3,10_WBerkeley 3 2661 2662,10,West Berkeley,...,0.0,0.0,0.0,0.0,"LINESTRING (563391.000 4189793.000, 563370.000...",NaN,NaN,NaN,non-FA,non-FA
3,0,10_WBerkeley,2662,2617,None,None,4,10_WBerkeley 4 2662 2617,10,West Berkeley,...,0.0,0.0,0.0,0.0,"LINESTRING (563370.000 4189980.000, 563206.000...",NaN,NaN,NaN,non-FA,non-FA
4,0,10_WBerkeley,2617,2618,None,None,5,10_WBerkeley 5 2617 2618,10,West Berkeley,...,0.0,0.0,0.0,0.0,"LINESTRING (563206.000 4190768.000, 562473.250...",NaN,NaN,NaN,non-FA,non-FA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41086,0,98_Red_SB,6123,6122,None,None,122,98_Red_SB 122 6123 6122,98,Regional Express ReX,...,0.0,0.0,0.0,0.0,"LINESTRING (569107.000 4148426.000, 568477.000...",NaN,NaN,NaN,non-FA,non-FA
41087,0,98_Red_SB,6122,6141,None,None,123,98_Red_SB 123 6122 6141,98,Regional Express ReX,...,0.0,0.0,0.0,0.0,"LINESTRING (568477.000 4148769.000, 568217.000...",NaN,NaN,NaN,non-FA,non-FA
41088,0,98_Red_SB,6141,6138,None,None,124,98_Red_SB 124 6141 6138,98,Regional Express ReX,...,0.0,0.0,0.0,0.0,"LINESTRING (568217.000 4148880.000, 567932.000...",NaN,NaN,NaN,non-FA,non-FA
41089,0,98_Red_SB,6138,6134,None,None,125,98_Red_SB 125 6138 6134,98,Regional Express ReX,...,0.0,0.0,0.0,0.0,"LINESTRING (567932.000 4149146.000, 567611.000...",NaN,NaN,NaN,non-FA,non-FA


In [23]:
# sum distances by line and tag_cat, for each time period

# groupby and sum distances
line_tag_dist_df = trn_links_merged[['NAME', 'A', 'B', 'MODE_TYPE', 'link_tag_cat', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']]
line_tag_dist_df = line_tag_dist_df.groupby(['MODE_TYPE', 'NAME', 'link_tag_cat'])[[
    'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']].sum().reset_index()
# line_tag_dist_df.rename(columns = {'DIST_EA': 'DIST_EA_line_tag',
#                                    'DIST_AM': 'DIST_AM_line_tag',
#                                    'DIST_MD': 'DIST_MD_line_tag',
#                                    'DIST_PM': 'DIST_PM_line_tag',
#                                    'DIST_EV': 'DIST_EV_line_tag'}, inplace=True)
display(line_tag_dist_df)

# pivot the dataframe to move tag_cat to columns
line_tag_dist_df = line_tag_dist_df.pivot_table(index=['MODE_TYPE', 'NAME'],
                                                columns=["link_tag_cat"],
                                                values=['DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV'], fill_value=0)
line_tag_dist_df.columns = ['_'.join(col).strip() for col in line_tag_dist_df.columns.values]
line_tag_dist_df.reset_index(inplace=True)
display(line_tag_dist_df)

,MODE_TYPE,NAME,link_tag_cat,DIST_EA,DIST_AM,DIST_MD,DIST_PM,DIST_EV
0,Commuter Rail,130_100NB,commuter rail,0.0,50.00,0.00,50.00,0.00
1,Commuter Rail,130_100SB,commuter rail,0.0,50.00,0.00,50.00,0.00
2,Commuter Rail,130_10NB,commuter rail,0.0,50.00,0.00,50.00,0.00
3,Commuter Rail,130_10SB,commuter rail,0.0,50.00,0.00,50.00,0.00
4,Commuter Rail,130_200PH8NB,commuter rail,0.0,50.00,0.00,50.00,0.00
...,...,...,...,...,...,...,...,...
1943,Local Bus,70_GG61-,non-FA,0.0,4.16,4.16,4.16,0.00
1944,Local Bus,70_GG68,non-FA,0.0,3.35,3.64,4.42,3.64
1945,Local Bus,70_GG68-,non-FA,0.0,4.42,3.88,4.42,2.05
1946,Local Bus,70_GG68A,non-FA,0.0,3.35,3.64,0.00,3.64


,MODE_TYPE,NAME,DIST_AM_FA,DIST_AM_commuter rail,DIST_AM_heavy rail,DIST_AM_non-FA,DIST_EA_FA,DIST_EA_commuter rail,DIST_EA_heavy rail,DIST_EA_non-FA,...,DIST_EV_heavy rail,DIST_EV_non-FA,DIST_MD_FA,DIST_MD_commuter rail,DIST_MD_heavy rail,DIST_MD_non-FA,DIST_PM_FA,DIST_PM_commuter rail,DIST_PM_heavy rail,DIST_PM_non-FA
0,Commuter Rail,130_100NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,50.0,0.0,0.00
1,Commuter Rail,130_100SB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,50.0,0.0,0.00
2,Commuter Rail,130_10NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,50.0,0.0,0.00
3,Commuter Rail,130_10SB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,50.0,0.0,0.00
4,Commuter Rail,130_200PH8NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,50.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Local Bus,70_GG61-,0.0,0.0,0.0,4.16,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,4.16,0.0,0.0,0.0,4.16
1309,Local Bus,70_GG68,0.0,0.0,0.0,3.35,0.0,0.0,0.0,0.0,...,0.0,3.64,0.0,0.0,0.0,3.64,0.0,0.0,0.0,4.42
1310,Local Bus,70_GG68-,0.0,0.0,0.0,4.42,0.0,0.0,0.0,0.0,...,0.0,2.05,0.0,0.0,0.0,3.88,0.0,0.0,0.0,4.42
1311,Local Bus,70_GG68A,0.0,0.0,0.0,3.35,0.0,0.0,0.0,0.0,...,0.0,3.64,0.0,0.0,0.0,3.64,0.0,0.0,0.0,0.00


In [24]:
# sum distances by line, for each time period

line_tag_df = trn_links_merged.groupby(['MODE_TYPE', 'NAME'])[['DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']].sum().reset_index()
display(line_tag_df)

,MODE_TYPE,NAME,DIST_EA,DIST_AM,DIST_MD,DIST_PM,DIST_EV
0,Commuter Rail,130_100NB,0.0,50.00,0.00,50.00,0.00
1,Commuter Rail,130_100SB,0.0,50.00,0.00,50.00,0.00
2,Commuter Rail,130_10NB,0.0,50.00,0.00,50.00,0.00
3,Commuter Rail,130_10SB,0.0,50.00,0.00,50.00,0.00
4,Commuter Rail,130_200PH8NB,0.0,50.00,0.00,50.00,0.00
...,...,...,...,...,...,...,...
1308,Local Bus,70_GG61-,0.0,4.16,4.16,4.16,0.00
1309,Local Bus,70_GG68,0.0,3.35,3.64,4.42,3.64
1310,Local Bus,70_GG68-,0.0,4.42,3.88,4.42,2.05
1311,Local Bus,70_GG68A,0.0,3.35,3.64,0.00,3.64


In [25]:
# calculate the shares of distances of each tag_cat, for each time period

# merge distance sums by tag_cat and by line only
line_df = pd.merge(line_tag_dist_df,
                   line_tag_df,
                   on=['MODE_TYPE', 'NAME'],
                   how='outer')
display(line_df)

# calculate shares
for i in ['EA', 'AM', 'MD', 'PM', 'EV']:
    for j in ['non-FA', 'FA', 'heavy rail', 'commuter rail']:
        line_df.loc[line_df['DIST_'+i] > 0, 'share_'+i+'_'+j] = line_df['DIST_'+i+'_'+j] / line_df['DIST_'+i]
        
display(line_df)

,MODE_TYPE,NAME,DIST_AM_FA,DIST_AM_commuter rail,DIST_AM_heavy rail,DIST_AM_non-FA,DIST_EA_FA,DIST_EA_commuter rail,DIST_EA_heavy rail,DIST_EA_non-FA,...,DIST_MD_non-FA,DIST_PM_FA,DIST_PM_commuter rail,DIST_PM_heavy rail,DIST_PM_non-FA,DIST_EA,DIST_AM,DIST_MD,DIST_PM,DIST_EV
0,Commuter Rail,130_100NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,50.0,0.0,0.00,0.0,50.00,0.00,50.00,0.00
1,Commuter Rail,130_100SB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,50.0,0.0,0.00,0.0,50.00,0.00,50.00,0.00
2,Commuter Rail,130_10NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,50.0,0.0,0.00,0.0,50.00,0.00,50.00,0.00
3,Commuter Rail,130_10SB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,50.0,0.0,0.00,0.0,50.00,0.00,50.00,0.00
4,Commuter Rail,130_200PH8NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,50.0,0.0,0.00,0.0,50.00,0.00,50.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Local Bus,70_GG61-,0.0,0.0,0.0,4.16,0.0,0.0,0.0,0.0,...,4.16,0.0,0.0,0.0,4.16,0.0,4.16,4.16,4.16,0.00
1309,Local Bus,70_GG68,0.0,0.0,0.0,3.35,0.0,0.0,0.0,0.0,...,3.64,0.0,0.0,0.0,4.42,0.0,3.35,3.64,4.42,3.64
1310,Local Bus,70_GG68-,0.0,0.0,0.0,4.42,0.0,0.0,0.0,0.0,...,3.88,0.0,0.0,0.0,4.42,0.0,4.42,3.88,4.42,2.05
1311,Local Bus,70_GG68A,0.0,0.0,0.0,3.35,0.0,0.0,0.0,0.0,...,3.64,0.0,0.0,0.0,0.00,0.0,3.35,3.64,0.00,3.64


,MODE_TYPE,NAME,DIST_AM_FA,DIST_AM_commuter rail,DIST_AM_heavy rail,DIST_AM_non-FA,DIST_EA_FA,DIST_EA_commuter rail,DIST_EA_heavy rail,DIST_EA_non-FA,...,share_MD_heavy rail,share_MD_commuter rail,share_PM_non-FA,share_PM_FA,share_PM_heavy rail,share_PM_commuter rail,share_EV_non-FA,share_EV_FA,share_EV_heavy rail,share_EV_commuter rail
0,Commuter Rail,130_100NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
1,Commuter Rail,130_100SB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
2,Commuter Rail,130_10NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
3,Commuter Rail,130_10SB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
4,Commuter Rail,130_200PH8NB,0.0,50.0,0.0,0.00,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Local Bus,70_GG61-,0.0,0.0,0.0,4.16,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1309,Local Bus,70_GG68,0.0,0.0,0.0,3.35,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1310,Local Bus,70_GG68-,0.0,0.0,0.0,4.42,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1311,Local Bus,70_GG68A,0.0,0.0,0.0,3.35,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0


In [26]:
# export line_df
line_df.to_csv('line_tag.csv', index=False)

In [27]:
print(list(line_df))

['MODE_TYPE', 'NAME', 'DIST_AM_FA', 'DIST_AM_commuter rail', 'DIST_AM_heavy rail', 'DIST_AM_non-FA', 'DIST_EA_FA', 'DIST_EA_commuter rail', 'DIST_EA_heavy rail', 'DIST_EA_non-FA', 'DIST_EV_FA', 'DIST_EV_commuter rail', 'DIST_EV_heavy rail', 'DIST_EV_non-FA', 'DIST_MD_FA', 'DIST_MD_commuter rail', 'DIST_MD_heavy rail', 'DIST_MD_non-FA', 'DIST_PM_FA', 'DIST_PM_commuter rail', 'DIST_PM_heavy rail', 'DIST_PM_non-FA', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV', 'share_EA_non-FA', 'share_EA_FA', 'share_EA_heavy rail', 'share_EA_commuter rail', 'share_AM_non-FA', 'share_AM_FA', 'share_AM_heavy rail', 'share_AM_commuter rail', 'share_MD_non-FA', 'share_MD_FA', 'share_MD_heavy rail', 'share_MD_commuter rail', 'share_PM_non-FA', 'share_PM_FA', 'share_PM_heavy rail', 'share_PM_commuter rail', 'share_EV_non-FA', 'share_EV_FA', 'share_EV_heavy rail', 'share_EV_commuter rail']


In [28]:
# line_df.loc[(line_df['share_AM_FA'] > 0) & (line_df['share_AM_FA'] < 1)][['share_AM_non-FA', 'share_AM_FA', 'share_AM_heavy rail', 'share_AM_commuter rail']]

In [29]:
# trn_links_merged = pd.merge(trn_links_merged, 
#                             trn_links_distance,
#                             on=['A', 'B', 'NAME'],
#                             how='left')

In [30]:
# trn_links_merged['DIST_mean'] = trn_links_merged[['DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']].mean(axis=1)

In [31]:
# trn_links_merged.loc[(~trn_links_merged.tag.isin(['freeway', 'arterial'])) & (trn_links_merged.tag.notnull())][[
#     'A', 'B', 'DISTANCE', 'Project', 'TollClass', 'tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV', 'DIST_mean','avg_distance'
# ]]

In [32]:
# trn_links_merged.loc[(~trn_links_merged.tag.isin(['freeway', 'arterial'])) & \
#                      (trn_links_merged.tag.notnull()) & \
#                      (trn_links_merged.DIST_mean == 0) & \
#                      (trn_links_merged.avg_distance != 0) & \
#                      (trn_links_merged.avg_distance.notnull())][[
#     'A', 'B', 'DISTANCE', 'Project', 'TollClass', 'tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV', 'DIST_mean','avg_distance'
# ]]

In [33]:
# trn_links_distance.shape[0]

In [34]:
# trn_links_distance[['A', 'B', 'NAME']].drop_duplicates().shape[0]

In [35]:
# trn_links_merged.loc[trn_links_merged.avg_distance.notnull()].MODE_TYPE.unique()

In [36]:
# # trnlink.csv
# df.loc[(df.A == 9875) & (df.B == 9876)][[
#     'A', 'B', 'name', 'time', 'mode', 'plot', 'stopA', 'stopB', 'distance','timeperiod']].sort_values(by='name')

In [37]:
# # network_links.dbf
# freeway_arterial_links_df.loc[(freeway_arterial_links_df.A == 9875) & (freeway_arterial_links_df.B == 9876)]

In [38]:
# # network_trn_links.shp
# trn_links_merged.loc[(trn_links_merged.A == 9875) & (trn_links_merged.B == 9876)][[
#     'A', 'B', 'DISTANCE', 'Project', 'TollClass', 'tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']]

In [39]:
# print(list(df))

In [40]:
# trn_links_merged.loc[(trn_links_merged.A == 38758) & (trn_links_merged.B == 2834)][['DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']]

In [41]:
# trn_links_merged['DIST_sum'] = trn_links_merged[['DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']].sum(axis=1)

In [42]:
# trn_links_merged.loc[(trn_links_merged['DIST_sum'] == 0) & (trn_links_merged['tag'].notnull())][[
#     'A', 'B', 'DISTANCE', 'Project', 'TollClass', 'tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']]

#### trn_route_links

In [43]:
trn_route_links_gdf = gpd.read_file(TRN_ROUTE_LINKS_FILE)
print('load {:,} rows of transit routes, {:,} unique A+B pair, the following columns: {}'.format(
    trn_route_links_gdf.shape[0],
    trn_route_links_gdf[['A', 'B']].drop_duplicates().shape[0],
    list(trn_route_links_gdf)))

print(trn_route_links_gdf.head())

load 36,401 rows of transit routes, 14,920 unique A+B pair, the following columns: ['Id', 'A', 'B', 'A_STATION', 'B_STATION', 'NAME_SET', 'MODE', 'MODE_NAME', 'MODE_TYPE', 'OPERATOR_T', 'LINE_COUNT', 'ROUTE_A_B', 'TRIPS_EA', 'TRIPS_AM', 'TRIPS_MD', 'TRIPS_PM', 'TRIPS_EV', 'PDCAP_EA', 'PDCAP_AM', 'PDCAP_MD', 'PDCAP_PM', 'PDCAP_EV', 'ABVOL_EA', 'ABVOL_AM', 'ABVOL_MD', 'ABVOL_PM', 'ABVOL_EV', 'LOAD_EA', 'LOAD_AM', 'LOAD_MD', 'LOAD_PM', 'LOAD_EV', 'PDSECAP_EA', 'PDSECAP_AM', 'PDSECAP_MD', 'PDSECAP_PM', 'PDSECAP_EV', 'LOADSE_EA', 'LOADSE_AM', 'LOADSE_MD', 'LOADSE_PM', 'LOADSE_EV', 'geometry']
   Id     A      B A_STATION B_STATION NAME_SET  MODE             MODE_NAME  \
0   0  1601   8838      None      None     56_5    56  Vacaville City Coach   
1   0  1601   8844      None      None     56_2    56  Vacaville City Coach   
2   0  1601   8844      None      None     56_5    56  Vacaville City Coach   
3   0  1604   8849      None      None     56_6    56  Vacaville City Coach   
4   0  160

In [44]:
trn_route_links_gdf.MODE_TYPE.value_counts(dropna=False)

Local Bus        22664
Express Bus      12673
Light Rail         702
Commuter Rail      136
Ferry              114
Heavy Rail         112
Name: MODE_TYPE, dtype: int64

In [45]:
trn_route_links_gdf.OPERATOR_T.value_counts(dropna=False)

Other          10337
AC_Transit      7279
GG_Transit      5600
SF_Muni         5386
SC_VTA          4629
SM_SamTrans     2957
BART             112
Caltrain          58
unknown_op        43
Name: OPERATOR_T, dtype: int64

In [46]:
# merge freeway and arterial nodes
trn_route_links_merged = trn_route_links_gdf.merge(freeway_arterial_links_df, on=['A', 'B'], how='left', indicator=True)
print(trn_route_links_merged['_merge'].value_counts())

left_only     30261
both           6140
right_only        0
Name: _merge, dtype: int64


In [47]:
# highlight 5 sets of links
freeway_idx = (trn_route_links_merged['_merge'] == 'both') & (trn_route_links_merged['link_tag'] == 'freeway')
arterial_tag_idx = (trn_route_links_merged['_merge'] == 'both') & (trn_route_links_merged['link_tag'] == 'arterial')
heavyRail_tag_idx = (trn_route_links_merged['_merge'] == 'left_only') & (trn_route_links_merged['MODE_TYPE'] == 'Heavy Rail')
commuterRail_tag_idx = (trn_route_links_merged['_merge'] == 'left_only') & (trn_route_links_merged['MODE_TYPE'] == 'Commuter Rail')

# trn_route_links_merged = trn_route_links_merged.loc[freeway_idx | arterial_tag_idx | heavyRail_tag_idx | lightRail_tag_idx | commuterRail_tag_idx]

# tag heavy rail, commuter rail and light rail links (links on freeway and arterial are already tagged)
trn_route_links_merged.loc[heavyRail_tag_idx, 'link_tag'] = 'heavy rail'
trn_route_links_merged.loc[commuterRail_tag_idx, 'link_tag'] = 'commuter rail'

# fill na
trn_route_links_merged['link_tag'].fillna('non-FA', inplace=True)

# create a more aggregated "tag category"
trn_route_links_merged['link_tag_cat'] = 'non-FA'
trn_route_links_merged.loc[trn_route_links_merged['link_tag'].isin(['freeway', 'arterial']), 'link_tag_cat'] = 'FA'
trn_route_links_merged.loc[trn_route_links_merged['link_tag'].isin(['heavy rail', 'commuter rail']), 'link_tag_cat'] = trn_route_links_merged['link_tag']

# drop '_merge'
trn_route_links_merged.drop(['_merge'], axis=1, inplace=True)

In [48]:
# export
trn_route_links_merged.to_file('trn_route_links_NGF_tag.shp')

C:\Users\ywang\AppData\Local\Temp\ipykernel_12820\926797315.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  trn_route_links_merged.to_file('trn_route_links_NGF_tag.shp')


#### trn_stops

In [49]:
trn_stops_gdf = gpd.read_file(TRN_STOPS_FILE)
trn_stops_gdf

,Id,LINE_NAME,STATION,N,SEQ,IS_STOP,BRD_EA,BRD_AM,BRD_MD,BRD_PM,BRD_EV,XIT_EA,XIT_AM,XIT_MD,XIT_PM,XIT_EV,geometry
0,0,100_ALAFB,Ferry Alameda Ferry Terminal,14604,1,1,0.0,2.21,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,POINT (562501.000 4182247.000)
1,0,100_ALAFB,Ferry SF Ferry Building,14600,2,1,0.0,0.00,0.0,0.0,0.0,0.0,2.21,0.0,0.0,0.0,POINT (553449.000 4183347.000)
2,0,100_ALAOAKFB,Ferry Alameda Ferry Terminal,14604,1,1,0.0,379.78,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,POINT (562501.000 4182247.000)
3,0,100_ALAOAKFB,Ferry Jack London Square Ferry Terminal,14605,2,1,0.0,2132.40,0.0,0.0,0.0,0.0,59.99,0.0,0.0,0.0,POINT (563719.000 4183438.000)
4,0,100_ALAOAKFB,Ferry SF Ferry Building,14600,3,1,0.0,0.00,0.0,0.0,0.0,0.0,2452.19,0.0,0.0,0.0,POINT (553449.000 4183347.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42574,0,98_Red_SB,None,6122,123,0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,POINT (568477.000 4148769.000)
42575,0,98_Red_SB,None,6141,124,0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,POINT (568217.000 4148880.000)
42576,0,98_Red_SB,None,6138,125,0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,POINT (567932.000 4149146.000)
42577,0,98_Red_SB,None,6134,126,1,0.0,46.00,0.0,24.0,0.0,0.0,324.00,34.0,92.0,0.0,POINT (567611.000 4149069.000)


In [50]:
# tag stops whose "LINE_NAME" is tagged
tagged_line_names = list(set(trn_links_merged.loc[trn_links_merged['link_tag'].notnull()]['NAME']))

trn_stops_gdf.loc[trn_stops_gdf['LINE_NAME'].isin(tagged_line_names), 'link_tag'] = 'tagged'

# export
trn_stops_gdf.to_file('trn_stops_NGF_tag.shp')

#### trn_lines

In [51]:
trn_lines_gdf = gpd.read_file(TRN_LINES_FILE)
trn_lines_gdf

,Id,NAME,NAME_SET,LONG_NAME,FREQ_EA,FREQ_AM,FREQ_MD,FREQ_PM,FREQ_EV,ONEWAY,...,SEATCAP_AM,SEATCAP_MD,SEATCAP_PM,SEATCAP_EV,PDSECAP_EA,PDSECAP_AM,PDSECAP_MD,PDSECAP_PM,PDSECAP_EV,geometry
0,0,100_ALAFB,100_ALAFB,None,0.0,120.0,0.0,0.0,0.0,1,...,400.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 553449.000..."
1,0,100_ALAOAKFB,100_ALAOAKFB,None,0.0,20.0,0.0,0.0,0.0,1,...,400.0,0.0,0.0,0.0,0.0,4800.0,0.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 563719.000..."
2,0,100_ALOAKSSF,100_ALOAK,None,0.0,60.0,0.0,0.0,0.0,1,...,250.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 563719.000..."
3,0,100_ALOKFBFW,100_ALOKFBFW,None,0.0,0.0,60.0,0.0,0.0,1,...,0.0,400.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,"LINESTRING (562501.000 4182247.000, 563719.000..."
4,0,100_ANMARSF,100_A,None,0.0,120.0,0.0,0.0,0.0,1,...,250.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,"LINESTRING (602471.000 4208298.000, 575476.000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,0,98_Blue_SB,98_Blue_,"""ReX Blue San Francisco to San Jose",0.0,10.0,20.0,10.0,0.0,1,...,39.0,39.0,39.0,0.0,0.0,936.0,585.0,936.0,0.0,"LINESTRING (552907.750 4182798.998, 553058.938..."
1367,0,98_Green_NB,98_Green_,"""ReX Green San Francisco Airport to",0.0,10.0,20.0,10.0,0.0,1,...,39.0,39.0,39.0,0.0,0.0,936.0,585.0,936.0,0.0,"LINESTRING (554134.000 4163399.000, 553684.000..."
1368,0,98_Green_SB,98_Green_,"""ReX Green Vallejo to San Francisco",0.0,10.0,20.0,10.0,0.0,1,...,39.0,39.0,39.0,0.0,0.0,936.0,585.0,936.0,0.0,"LINESTRING (564643.000 4217267.000, 565295.000..."
1369,0,98_Red_NB,98_Red_,"""ReX Red Redwood City to Oakland No",0.0,10.0,20.0,10.0,0.0,1,...,39.0,39.0,39.0,0.0,0.0,936.0,585.0,936.0,0.0,"LINESTRING (567746.000 4149040.000, 567611.000..."


In [52]:
print(list(trn_lines_gdf))

['Id', 'NAME', 'NAME_SET', 'LONG_NAME', 'FREQ_EA', 'FREQ_AM', 'FREQ_MD', 'FREQ_PM', 'FREQ_EV', 'ONEWAY', 'MODE', 'MODE_NAME', 'MODE_TYPE', 'OPERATOR_T', 'FIRST_N', 'FIRST_NAME', 'LAST_N', 'LAST_NAME', 'N_OR_S', 'E_OR_W', 'VEHTYPE_EA', 'VEHTYPE_AM', 'VEHTYPE_MD', 'VEHTYPE_PM', 'VEHTYPE_EV', 'VEHCAP_EA', 'VEHCAP_AM', 'VEHCAP_MD', 'VEHCAP_PM', 'VEHCAP_EV', 'PDCAP_EA', 'PDCAP_AM', 'PDCAP_MD', 'PDCAP_PM', 'PDCAP_EV', 'SEATCAP_EA', 'SEATCAP_AM', 'SEATCAP_MD', 'SEATCAP_PM', 'SEATCAP_EV', 'PDSECAP_EA', 'PDSECAP_AM', 'PDSECAP_MD', 'PDSECAP_PM', 'PDSECAP_EV', 'geometry']


In [53]:
trn_lines_gdf.shape[0]

1371

In [54]:
trn_lines_gdf[['NAME']].drop_duplicates().shape[0]

1371

In [55]:
chk = pd.merge(trn_lines_gdf, line_df, on='NAME', how='outer', indicator=True)

In [56]:
chk['_merge'].value_counts()

both          1313
left_only       58
right_only       0
Name: _merge, dtype: int64

In [57]:
chk.loc[chk['_merge'] == 'left_only']

,Id,NAME,NAME_SET,LONG_NAME,FREQ_EA,FREQ_AM,FREQ_MD,FREQ_PM,FREQ_EV,ONEWAY,...,share_MD_commuter rail,share_PM_non-FA,share_PM_FA,share_PM_heavy rail,share_PM_commuter rail,share_EV_non-FA,share_EV_FA,share_EV_heavy rail,share_EV_commuter rail,_merge
0,0,100_ALAFB,100_ALAFB,None,0.0,120.0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,0,100_ALAOAKFB,100_ALAOAKFB,None,0.0,20.0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0,100_ALOAKSSF,100_ALOAK,None,0.0,60.0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0,100_ALOKFBFW,100_ALOKFBFW,None,0.0,0.0,60.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0,100_ANMARSF,100_A,None,0.0,120.0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,0,100_APFB,100_APFB,None,0.0,30.0,60.0,30.0,60.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,0,100_BERKFB,100_BE,None,0.0,30.0,60.0,30.0,60.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7,0,100_FBALA,100_FBALA,None,0.0,240.0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,0,100_FBALAOAK,100_FBALAOAK,None,0.0,20.0,0.0,0.0,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,0,100_FBAP,100_FBAP,None,0.0,30.0,60.0,30.0,60.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [58]:
trn_links_gdf = gpd.read_file(TRN_LINKS_FILE)
chk = chk.merge(trn_links_gdf[['NAME', 'MODE_TYPE']], on='NAME', how='left')

In [59]:
chk.loc[chk['_merge'] == 'left_only'].MODE_TYPE.unique()

array(['Ferry'], dtype=object)

In [60]:
# link_filename = os.path.join(r"L:\Application\Model_One\NextGenFwys\Scenarios\2035_TM152_NGF_NP02_BPALTsegmented_00\OUTPUT\trn\trnlink.csv")

# trnlink_df_raw = pd.read_csv(link_filename)
# print("Read {} rows from {}".format(len(trnlink_df_raw), link_filename))

In [61]:
# # filter to just these cols
# trnlink_df2 = trnlink_df_raw[["A","B","mode","name","time","distance","AB_VOL","source"]]
# print("\n{}".format(trnlink_df2.head()))

# # source = 'trnlinkea_wlk_loc_wlk.csv' - pull timeperiod
# # group by A,B,mode,timeperiod -- so rollup across source for the timeperiod
# trnlink_df2['timeperiod'] = trnlink_df2.source.str[7:9]
# trnlink_df = trnlink_df2.groupby(["A","B","mode","name","timeperiod"]).agg({"AB_VOL" :numpy.sum,
#                                                                            "distance":numpy.mean,
#                                                                            "time"    :numpy.mean})
# trnlink_df.reset_index(drop=False, inplace=True)
# print("Aggregated to {} links across time periods".format(len(trnlink_df)))

In [62]:
# trnlink_df2

In [63]:
# # move timeperiods to columns
# trnlink_df = pd.pivot_table(trnlink_df, index=["A","B","name","mode"], columns=["timeperiod"], values=["AB_VOL","distance","time"])
# trnlink_df.columns = ["_".join(col).strip() for col in trnlink_df.columns.values]
# trnlink_df.fillna(value=0, inplace=True)
# print("Pivoted to {} links".format(len(trnlink_df)))

# display("\n{}".format(trnlink_df.head()))

# # index-oriented records, so A,B,name,mode => dict
# trnlink_recs = trnlink_df.to_dict(orient="index")
# print(trnlink_recs)

In [64]:
# trnlink_recs

In [65]:
# trn_links_merged.loc[(trn_links_merged['DIST_AM'] == 0) & \
#                      (trn_links_merged['DIST_EA'] == 0) & \
#                      (trn_links_merged['DIST_MD'] == 0) & \
#                      (trn_links_merged['DIST_PM'] == 0) & \
#                      (trn_links_merged['DIST_EV'] == 0) & \
#                      (trn_links_merged['tag'].notnull())][[
#     'A', 'B', 'NAME', 'DISTANCE', 'Project', 'TollClass', 'tag', 'DIST_EA', 'DIST_AM', 'DIST_MD', 'DIST_PM', 'DIST_EV']]

In [66]:
# print(list(trn_route_links_gdf))

In [67]:
# trn_links_gdf.loc[(trn_links_gdf.A == 6989) & \
#                         (trn_links_gdf.B == 13655)]

In [68]:
# trnlink_df2.loc[(trnlink_df2.name == '110_E_OUT')]

In [69]:
# transit_crowding_df.loc[(transit_crowding_df.NAME == '110_E_OUT') & \
#                         (transit_crowding_df.A == 6989) & \
#                         (transit_crowding_df.B == 13655)]

In [70]:
# transit_crowding_lines = []  # list of line names
# crowding_line_dict     = {}  # name -> {(veh_type_updated|seatcap|period_seatcap|period_standcap, EA|AM|MD|PM|EV) -> value}
# crowding_link_dict     = {}  # (name,a,b,seq) -> {(load_seatcap|load_standcap|AB_VOL|AB_BRDA|AB_XITB, EA|AM|MD|PM|EV) -> value}

# TRANSIT_CROWDING_FILE = r'L:\Application\Model_One\NextGenFwys\Scenarios\2035_TM152_NGF_NP02_BPALTsegmented_00\OUTPUT\metrics\transit_crowding_complete.csv'
# transit_crowding_df = pd.read_csv(TRANSIT_CROWDING_FILE)
# transit_crowding_lines = transit_crowding_df["NAME"].drop_duplicates().tolist()
# print("Read {} rows and {} unique line names from {}".format(len(transit_crowding_df), len(transit_crowding_lines), TRANSIT_CROWDING_FILE))

In [71]:
# transit_crowding_df[['A', 'B', 'NAME']]

In [72]:
# # Pivot and make into a lookup for line attributes
# crowding_line_df = transit_crowding_df[["NAME","period","veh_type_updated","seatcap","period_seatcap","standcap","period_standcap"]]
# crowding_line_df = crowding_line_df.drop_duplicates().pivot_table(index=["NAME"], columns=["period"], 
#                                                                   values=["veh_type_updated","seatcap","period_seatcap","standcap","period_standcap"],
#                                                                   aggfunc='first', fill_value=0)
# print("crowding_line_df head=\n{}".format(crowding_line_df.head(20)))
# crowding_line_dict = crowding_line_df.to_dict(orient='index')

# # need AB_XITA for stops -- so create it by joining with next link
# ab_xita_df = transit_crowding_df[["NAME","period","SEQ","AB_XITB"]].copy()  # these exits occurred at B -- shift to A
# ab_xita_df["SEQ"] = ab_xita_df["SEQ"] + 1
# ab_xita_df.rename(columns={"AB_XITB":"AB_XITA"}, inplace=True)
# transit_crowding_df = pandas.merge(left=transit_crowding_df, right=ab_xita_df, how="left", on=["NAME","period","SEQ"]).fillna(value={"AB_XITA":0})

# # Pivot and make into a lookup for link attributes
# crowding_link_df = transit_crowding_df[["NAME","A","B","SEQ","period","load_seatcap","load_standcap","AB_VOL","AB_BRDA","AB_XITB","AB_XITA"]].pivot_table(
#                                                                   index=["NAME","A","B","SEQ"], columns=["period"], fill_value=0)
# print("crowding_link_df head=\n{}".format(crowding_link_df.head(20)))
# crowding_link_dict = crowding_link_df.to_dict(orient='index')